# Load the data

In [1]:
from result_saver import SaverProvider

provider = SaverProvider()

In [2]:
from Scratch import metadata_loader
import numpy as np
from tqdm import tqdm

DEVICE = "ibmq_mumbai"
LOGICAL = 0
DISTANCE = 3
ROUNDS = 3

# Load the metadata
md = metadata_loader(True, True)
md = md[md["job_status"] == "JobStatus.DONE"]
md = md[md["code"] == "RepetitionCodeCircuit"]
md = md.dropna(subset=["rounds"])
md = md[md["meas_level"] == 1]
md['rounds'] = md['rounds'].astype(int)
md['distance'] = md['distance'].astype(int)

md = md[md["backend_name"] == DEVICE]
md = md[md["logical"] == LOGICAL]
md = md[md["rounds"] == DISTANCE]
md = md[md["distance"] == ROUNDS]
md

,creation_date,notebook_name,backend_name,job_id,tags,meas_level,shots,tags_xp,rep_delay,sampled_state,...,optimization_level,code,distance,rounds,logical,layout,descr,num_CNOTS_layers,num_CNOT_layers,resets
2607,2024-03-10 18:36:49.567000+01:00,mean_pos_mumbai,ibmq_mumbai,cqpyzcd2b6ng008y69pg,[RepCode Mean Positions],1.0,6944.0,NaN,0.0005,NaN,...,NaN,RepetitionCodeCircuit,3,3,0,NaN,Mean positions,NaN,NaN,False


# Analyze the job

In [3]:
job = provider.retrieve_job(md.iloc[0]["job_id"])

In [4]:
print(len(job.initial_layouts()))
layout = job.initial_layouts()[8]
print(layout)
print(job.deserialize_layout(layout))


16
{"Qubit(QuantumRegister(22, 'ancilla'), 0)": '0', "Qubit(QuantumRegister(2, 'link_qubit'), 1)": '1', "Qubit(QuantumRegister(3, 'code_qubit'), 1)": '2', "Qubit(QuantumRegister(2, 'link_qubit'), 0)": '3', "Qubit(QuantumRegister(3, 'code_qubit'), 2)": '4', "Qubit(QuantumRegister(3, 'code_qubit'), 0)": '5', "Qubit(QuantumRegister(22, 'ancilla'), 1)": '6', "Qubit(QuantumRegister(22, 'ancilla'), 2)": '7', "Qubit(QuantumRegister(22, 'ancilla'), 3)": '8', "Qubit(QuantumRegister(22, 'ancilla'), 4)": '9', "Qubit(QuantumRegister(22, 'ancilla'), 5)": '10', "Qubit(QuantumRegister(22, 'ancilla'), 6)": '11', "Qubit(QuantumRegister(22, 'ancilla'), 7)": '12', "Qubit(QuantumRegister(22, 'ancilla'), 8)": '13', "Qubit(QuantumRegister(22, 'ancilla'), 9)": '14', "Qubit(QuantumRegister(22, 'ancilla'), 10)": '15', "Qubit(QuantumRegister(22, 'ancilla'), 11)": '16', "Qubit(QuantumRegister(22, 'ancilla'), 12)": '17', "Qubit(QuantumRegister(22, 'ancilla'), 13)": '18', "Qubit(QuantumRegister(22, 'ancilla'), 14)

In [5]:
print(job.result().get_memory(11).shape)

(6944, 9)


# Decode

In [6]:
import json

import pymatching
import stim 
from soft_info import get_noise_dict_from_backend, get_avgs_from_dict, get_repcode_IQ_map, create_or_load_kde_grid
from src import cpp_soft_info
from Scratch import load_calibration_memory

_DETAILED = False

rel_error = 1
_RESETS = False

# KDE BANDWIDTHS
lin = [0.4, 0.7, 20]
num_points = 51
lin = [0.4, 0.7, 2]
num_points = 2

bandwidths = np.linspace(lin[0], lin[1], lin[2])

# bandwidths = [0.6]
num_points = 4


# Get the date and noise
date = md.iloc[0]["creation_date"]
noise_dict = get_noise_dict_from_backend(provider, DEVICE, date=date)

# Get the KDEs
all_memories = load_calibration_memory(provider, md.iloc[0]["job_id"])

kde_dict = cpp_soft_info.get_KDEs(all_memories, bandwidths, relError=1, absError=-1, num_points=num_points)

# Get kde grids
# grid_dict, scaler_dict = create_or_load_kde_grid(provider, md.iloc[0]["job_id"], num_grid_points=10, num_std_dev=2)

18:36:41 Warning: Could not get two gate error of ECR due to 'Could not find the desired property for ecr', taking CX instead.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 49
Specified job execution date: 2024-03-10 18:37:54.874642+01:00
Found jobs for backend ibmq_mumbai with closest execution date 2024-03-10 16:56:34.510733+00:00.


In [8]:
import time 

layouts = list(job.initial_layouts())

num_errors = []
for idx, layout_dict in tqdm(enumerate(layouts[:]), desc="Decoding circuits"):

    layout_dict_des = job.deserialize_layout(layout_dict)

    layout = list(layout_dict_des['link_qubit'].values()) + list(layout_dict_des['code_qubit'].values())

    qubit_mapping = get_repcode_IQ_map(layout, synd_rounds=ROUNDS)
    avgs = get_avgs_from_dict(noise_dict, layout)

    # Initialize the stim model
    circuit = stim.Circuit.generated("repetition_code:memory",
                            distance=DISTANCE,
                            rounds=ROUNDS,
                            after_clifford_depolarization=avgs["two_gate"], #two-qubit-fidelity,
                            after_reset_flip_probability=0 if not _RESETS else avgs["readout"], #reset error
                            before_measure_flip_probability=avgs["readout"], #measurement error,
                            before_round_data_depolarization=avgs["idle"]) #idle error)
    model = circuit.detector_error_model(decompose_errors=False)

    memory = job.result().get_memory(idx)
    result_kde = cpp_soft_info.decode_IQ_kde(model, memory, ROUNDS, int(LOGICAL), _RESETS, 
                                                qubit_mapping, kde_dict, _DETAILED, relError=rel_error, absError=-1)
    
    num_errors.append(result_kde.num_errors)
    print(f"num_errors: {result_kde.num_errors} out of {len(memory)}")

print(f"num_errors: {num_errors}")

Decoding circuits: 0it [00:00, ?it/s]

Decoding circuits: 1it [00:01,  1.48s/it]

num_errors: 10 out of 6944


Decoding circuits: 2it [00:02,  1.44s/it]

num_errors: 16 out of 6944


...

num_errors: 30 out of 6944


Decoding circuits: 15it [00:20,  1.37s/it]

num_errors: 12 out of 6944


Decoding circuits: 16it [00:22,  1.39s/it]

num_errors: 59 out of 6944
num_errors: [10, 16, 19, 30, 33, 22, 54, 9, 40, 5, 34, 16, 47, 30, 12, 59]


# Performance analysis

import cProfile

def f():
    num_errors = []
    for idx, layout_dict in tqdm(enumerate(job.initial_layouts()), desc="Decoding circuits"):

        layout_dict_des = job.deserialize_layout(layout_dict)

        layout = list(layout_dict_des['link_qubit'].values()) + list(layout_dict_des['code_qubit'].values())

        qubit_mapping = get_repcode_IQ_map(layout, synd_rounds=3)
        avgs = get_avgs_from_dict(noise_dict, layout)

        # Initialize the stim model
        circuit = stim.Circuit.generated("repetition_code:memory",
                                distance=3,
                                rounds=3,
                                after_clifford_depolarization=avgs["two_gate"], #two-qubit-fidelity,
                                after_reset_flip_probability=0 if not _RESETS else avgs["readout"], #reset error
                                before_measure_flip_probability=avgs["readout"], #measurement error,
                                before_round_data_depolarization=avgs["idle"]) #idle error)
        model = circuit.detector_error_model(decompose_errors=False)

        memory = job.result().get_memory(idx)
        result_kde = cpp_soft_info.decode_IQ_kde(model, memory, 3, int(LOGICAL), _RESETS, 
                                                    qubit_mapping, kde_dict, _DETAILED, relError=rel_error, absError=-1,
                                                    nb_intervals=-1)
        
        num_errors.append(result_kde.num_errors)


cProfile.run("f()")
